In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [3]:
batch_size = 50
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Before you can train your model, 
sequences = []
with open("BI_scFvFc_mimic_seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            sequences.append(seq)
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

In [5]:
## 
average_hidden_list = []
final_hidden_list = []
hs_list = []
final_cell_list = []


num2 = range(0, 50)
x = 0
y = 50
for i in num2:
    num1 = range(x, y)
    for j in num1:
        avg_hidden, final_hidden, final_cell, hs_out = (b.get_rep_hs(sequences[j]))
        average_hidden_list.append(avg_hidden)
        final_hidden_list.append(final_hidden)
        final_cell_list.append(final_cell)
        hs_list.append(hs_out)
    print('rep')
    x = x + 50
    y = y + 50
    


rep
rep
rep


IndexError: list index out of range

In [6]:
average_hidden_pd = pd.DataFrame(np.row_stack(average_hidden_list))
final_hidden_pd = pd.DataFrame(np.row_stack(final_hidden_list))
hidden_state = pd.DataFrame(np.row_stack(hs_list))
print(hidden_state)

             0         1         2         3         4         5         6   \
0     -0.165004  0.367741  0.358379 -0.652705  0.310745  0.112354  0.319343   
1     -0.144551  0.022954 -0.006241 -0.944919 -0.200301 -0.100880  0.144920   
2     -0.129126  0.087369 -0.026372 -0.981263 -0.036077 -0.069358  0.142709   
3     -0.090009  0.123185 -0.039344 -0.984940  0.064744 -0.085909  0.121372   
4     -0.068163  0.175416 -0.049245 -0.972869  0.111550 -0.115232  0.078261   
5     -0.031792  0.126204 -0.057019 -0.974423  0.074448 -0.179573 -0.010163   
6     -0.064345  0.139282 -0.040143 -0.981826  0.035730 -0.203455 -0.004673   
7     -0.023996  0.076038 -0.047925 -0.985140  0.050574 -0.213008  0.015928   
8     -0.091324  0.033268 -0.045600 -0.973642  0.003916 -0.198281  0.057883   
9     -0.119171  0.101985 -0.047136 -0.987462 -0.017699 -0.157126  0.037545   
10    -0.059051  0.071283 -0.071183 -0.986316 -0.049865 -0.099968  0.007359   
11     0.008425  0.077544 -0.067515 -0.990571 -0.047

In [7]:
average_hidden_pd.to_csv("BI_scFvFc_mimic_reps.csv")
final_hidden_pd.to_csv("BI_scFvFc_mimic_finalhidden.csv")

In [9]:
import pickle
save_loc = "C:\\Users\\pkinn\\Documents\\UniRep\\full representations\\emi larger set\\"
data_name = 'emi_novel_clones_reps'
file_append = '.pickle'


fn = save_loc + data_name + 'avg_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(average_hidden_list, f)

fn = save_loc + data_name + 'final_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_hidden_list, f)

fn = save_loc + data_name + 'final_cell' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_cell_list, f)
   
fn = save_loc + data_name + 'hidden_state' + file_append
with open(fn, 'wb') as f:
    pickle.dump(hs_list, f)
   
fn = save_loc + data_name + 'all_output_hs' + file_append
with open(fn, 'wb') as f:
    pickle.dump([average_hidden_list, final_hidden_list, final_cell_list, hs_list], f)